# Text Polarity Classification

## Import and preprocessing

In [1]:
# Connect drive
from google.colab import drive
drive.mount('/content/drive')

# Text preprocessing and imports
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

import pandas as pd
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import tensorflow as tf
import torch

# Data augmentation
!pip install nlpaug
import nlpaug.augmenter.word as naw

# Finetuning

!pip install transformers
!pip install simpletransformers
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from simpletransformers.classification import ClassificationModel, ClassificationArgs

Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 8.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

### GPU Colab

In [2]:
# Check if a GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
!nvidia-smi

Thu Apr 27 14:11:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      2MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Upload training and test

In [3]:
train = pd.read_csv('/content/drive/MyDrive/NCU/Data Science and Machine Learning/Assignment 3/Datasets/train.csv')
train = train.drop(train.columns[0], axis=1)

valid = pd.read_csv('/content/drive/MyDrive/NCU/Data Science and Machine Learning/Assignment 3/Datasets/test.csv')
valid = valid.drop(valid.columns[0], axis=1)

- GPT2 set

In [4]:
aug_train = pd.read_csv('/content/drive/MyDrive/NCU/Data Science and Machine Learning/Assignment 3/Datasets/GPT2_train.csv')

In [5]:
shuffled = aug_train.sample(frac=1).reset_index(drop=True)

In [6]:
# THIS IS THE DATASET THAT WILL BE PROCESSED, AUGMENTED (IF NEEDED) AND TRAINED
train = shuffled

### Preprocessing (stopwords and punctuation removal, lemmatizing)

- Preprocess text

In [7]:
def preprocess_text(text):
    # convert text to lowercase
    text = text.lower()
    # remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # tokenize the text into words
    words = nltk.word_tokenize(text)
    # remove stop words
    words = [word for word in words if word not in stopwords.words('english')]
    # lemmatize the remaining words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    # join the words back into a single string
    text = " ".join(words)
    return text

- Data augmentation

In [8]:
def augment_data(df, n_augmentations):
    augmented = pd.DataFrame(columns=df.columns)
    augmenter = naw.SynonymAug(aug_src='wordnet')

    for i, row in df.iterrows():
        text = row['TEXT']
        for j in range(n_augmentations):
            augmented_text = augmenter.augment(text)
            if isinstance(augmented_text, list):
                augmented_text = ' '.join(augmented_text)
            augmented.loc[len(augmented)] = [augmented_text, row['LABEL']]

    print(f'Augmentation number: {n_augmentations}')
    
    return pd.concat([df, augmented]).reset_index(drop=True)


- Apply augmentation and preprocess

In [9]:
def text_valid_prepr(train_df, valid_df, n_augmentations=0):

    # Increase training data
    print(f'Increasing training set {n_augmentations} times...')

    train_aug = augment_data(train_df, n_augmentations)

    # Process text sentences
    print(f'The training set has now {len(train_aug["TEXT"])} rows')
    print('Preprocessing text...')

    train_aug['TEXT'] = train_aug['TEXT'].apply(preprocess_text)

    print('Preprocessing the validation data...')

    valid_df = [sent for sent in valid_df['TEXT']]

    print('Preprocessing finished!')

    return train_aug, valid_df

train, valid = text_valid_prepr(train, valid, 0)

Increasing training set 0 times...
Augmentation number: 0
The training set has now 6000 rows
Preprocessing text...
Preprocessing the validation data...
Preprocessing finished!


- Calculating max_length

In [10]:
def max_length(train_df):

    train_list = [sent for sent in train['TEXT']]
    
    token_list = []
    max_length = 0

    for sent in train_list:
      sentence = nltk.word_tokenize(sent)
      count = len(sentence)

      if max_length < count:
        max_length = count

    return max_length

print(max_length(train))

27


## Finetuning XLNet

### Change hyperparameters of the transformer

In [14]:
trainer_args = {
   'model_type':  'xlnet',
   'model_name': 'xlnet-base-cased',
   'output_dir': 'outputs/',
   'cache_dir': 'cache/',
   'fp16': True,
   'fp16_opt_level': 'O1',
   'max_seq_length': 40,
   'train_batch_size': 32,
   'eval_batch_size':8,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 7,
   'weight_decay': 0,
   'learning_rate': 2e-5,
   'adam_epsilon': 1e-8, #0.01 = 1 * 10^-2 = 1e-2
   'warmup_ratio': 0.08,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,
   'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,
   'overwrite_output_dir': True,
   'reprocess_input_data': False,
}

### Train

In [12]:
def train_eval(train_df, valid_df, n_submission):

  model = ClassificationModel(
      'xlnet', 
      'xlnet-base-cased', 
      args = trainer_args,
      )
  
  print("training the model...")

  model.train_model(train_df)

  print("model trained!")
  print(" ")
  print("running predictions...")
  
  predictions, raw_outputs = model.predict(valid_df)

  submission_csv = pd.DataFrame(predictions, columns = ['LABEL'])

  submission_csv.to_csv (f'submission_{n_submission}.csv', index=True, header=True)

In [15]:
submission = '2Gaug_XLNet_9_6'
train_eval(train, valid, submission)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

training the model...


/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Epoch:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 0 of 7:   0%|          | 0/188 [00:00<?, ?it/s]

...

Running Epoch 5 of 7:   0%|          | 0/188 [00:00<?, ?it/s]

Running Epoch 6 of 7:   0%|          | 0/188 [00:00<?, ?it/s]

model trained!
 
running predictions...


  0%|          | 0/11000 [00:00<?, ?it/s]

  0%|          | 0/1375 [00:00<?, ?it/s]